In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## State of experiment

The experiment is not yet finished, some measures should be made to evaluate the correctness of predictions. Looking at them as is, the classification scores would  very often place read as first choice for many rules. Might not be adapted and for now we stick to the cosine similarity pipeline.

In [ ]:
from transformers import pipeline
from pygaur.gnlp import get_df_tags_keywords, load_extracted_file
import pandas as pd

import re

/nix/store/qs3hz3vg5l7blv6bgg4qjnw2ilx9m4rw-python3-3.11.11-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import data

In [ ]:
action_tag_path = "/home/gquetel/repos/gaur/pygaur/pygaur/data/mysql/tags/actions.tags"
df_actions = get_df_tags_keywords(action_tag_path)
labels_actions = list(set(df_actions['tag'].values))


In [ ]:
extracted_filepath = (
    "/home/gquetel/repos/gaur/pygaur/pygaur/data/mysql/extracted/mysql.gaur"
)
df_extracted = load_extracted_file(extracted_filepath)


def construct_documents(df_ex: pd.DataFrame) -> pd.DataFrame:
    index = df_ex.index.values
    documents = (
        # Remove the suffix part of the rule identifier.
        df_ex.index.to_series().apply(lambda x: re.sub(r"\.\d+$", "", str(x)))
        + " "
        + df_ex["terminals"].astype(str)
        + df_ex["code"].astype(str)
    )
    df_docs = pd.DataFrame(documents, columns=["doc"], index=index)

    df_docs["doc"] = df_docs["doc"].str.replace("_", " ")
    df_docs["doc"] = df_docs["doc"].str.lower()

    return df_docs

df_docs = construct_documents(df_extracted)


In [ ]:
df_docs = df_docs.head(100)      

In [ ]:
pipe = pipeline(model='facebook/bart-large-mnli')
p =  pipe(df_docs['doc'].to_list(), candidate_labels=labels_actions)
for rule_name, result in zip(df_docs.index.values, p):
    print(rule_name, result["labels"][0], result["scores"][0])

KeyboardInterrupt: 